In [2]:
import os
import json
import jsonlines

import torch
import ctypes
import gc
import random

In [ ]:
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
clean_memory()

In [ ]:
# metadata
# EN
# len: 4459532
# key per line: dict_keys(['meta', 'text', 'input', 'output', 'instruction'])
# 'meta' key: dict_keys(['Dataset', 'Gen', 'IFT', 'Lang', 'CFT-MR', 'CFT-P', 'CFT-SR', 'Task', 'original_path'])

In [ ]:
# {'ShareGPT', 'belle_cn', 'instinwild', 'Chinese-medical', 'alpacaGPT4', 'COIG', 'HC3', 'MOSS'}

In [ ]:
import jsonlines

data_stat_path = "/home/chenxiaojia/data/competition/competition_kit/data-juicer/outputs/v6_text/en_processed_stats.jsonl"
data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_keep_long_token_perplexity_refine_v6_en_2023-11-06-01-30-32/data/training_dataset.jsonl"
output = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_keep_long_token_perplexity_refine_v6_en_2023-11-06-01-30-32/data/flitered_training_dataset.jsonl"

cnt = 0
new_data_writer = jsonlines.open(output, 'w')

with jsonlines.open(data_stat_path) as file1, jsonlines.open(data_path) as file2:
    for data1, data2 in zip(file1, file2):
        # process data from both files here
        data1 = data1["__dj__stats__"]
        if data1["flagged_words_ratio"] <= 0.017:
            # print(data1["flagged_words_ratio"])
            # print(data2)
            # print("")
            cnt += 1
            new_data_writer.write(data2)

new_data_writer.close()

In [ ]:
import json
import jsonlines

# data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_all_3sigma_v2_en_2023-11-10-10-38-23/data/en/datasets_en.jsonl"
# data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_all_3sigma_v2_en_2023-11-10-10-38-23/data/en/datasets_en.jsonl"
data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/data/raw_data/raw_data_en.jsonl"

dataset_type = {}
task_type = {}
reader = jsonlines.open(data_path, 'r')

new_r = jsonlines.open("/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/data/debug_dj/debug_1k", 'w')

cnt = 0
for line in reader:
    if cnt <= 1000:
        new_r.write(line)
        cnt += 1
new_r.close()
# for line in reader:
#     ds_type = line['meta']['Dataset']
#     tsk_type = line['meta']['Task']
#     if dataset_type.get(ds_type, -1) == -1:
#         dataset_type[ds_type] = 0
#         task_type[tsk_type] = 0
#     dataset_type[ds_type] += 1
#     task_type[tsk_type] += 1



In [ ]:
# for k, v in dataset_type.items():
#     print(k, len(v))

print(task_type)
print(dataset_type)

In [ ]:
import json

with open('temp_data/sharegpt.json', 'w') as f:
    json.dump(dataset_type['ShareGPT'], f)
    


In [ ]:
f = json.load(open('temp_data/sharegpt.json', 'r'))

## LLM for evaluation data quality

In [ ]:
# ernie
import erniebot

erniebot.api_type = 'aistudio'
erniebot.access_token = 'a335b39380dbc8aaaec76f78725d1dc867cf2d27'

models = erniebot.Model.list()
print(models)



In [ ]:
test_text = "We would like to request your feedback on the performance of AI assistant in response to the instruction and the given input displayed following.\nInstruction: [where did all the money that was lost in the sub prime mortgage crisis and subsequent global recession actually go ? Money does n't simply disappear so if people lost money then it must have gone somewhere . Where ? Explain like I'm five.]\nInput: []\nResponse: [The subprime mortgage crisis was a financial crisis that occurred in the late 2000s as a result of the collapse of the housing market. Many people had taken out mortgages to buy homes, but when the housing market crashed, the value of their homes decreased and they were no longer able to make their mortgage payments. This led to a wave of foreclosures, which is when the bank takes back a home because the owner can't make the payments. As a result, many banks and financial institutions lost a lot of money because they had invested in these mortgages and were now unable to recoup their investments. This caused a financial crisis, as the banks and financial institutions that had lost money were no longer able to lend money to other people or businesses. The money that was lost during the financial crisis did not simply disappear. Instead, it was lost by the banks and financial institutions that had invested in the subprime mortgages and were unable to recoup their investments when the housing market crashed. This loss of money had a ripple effect, as the banks and financial institutions were then unable to lend money to other people or businesses, which slowed down the economy and led to a global recession.]\nPlease rate according to the accuracy of the response to the instruction and the input. Each assistant receives a score on a scale of 0 to 5, where a higher score indicates higher level of the helpfulness. Please first output a single line containing the value indicating the scores."
num_tokens = erniebot.utils.token_helper.approx_num_tokens(test_text)
num_tokens

In [ ]:
response = erniebot.ChatCompletion.create(
    model='ernie-bot',
    messages=[{'role': 'user', 'content': test_text,
    }],
)

In [ ]:
response.result

## Falcon-1B test

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_all_3sigma_v4_en_2023-11-11-17-37-38"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).cuda()
model = model.eval()


In [ ]:
text = "model = tf.keras.Sequential([\n layers.Embedding(num\\_words, embedding\\_dim, embeddings\\_initializer=tf.keras.initializers.Constant(embedding\\_matrix),\n input\\_length=max\\_length, trainable=False),\n layers.Conv1D(128, 5, activation='relu'),\n layers.MaxPooling1D(pool\\_size=4),\n layers.Flatten(),\n layers.Dense(1, activation='sigmoid')\n])\n\nwe have used this model with glove embedding to perform classification..can you explain the model and what each layer of the model does...our accuracy is 84 right now so how do we improve the accuracy by enhancing the model"
gen_kwargs = {"max_length": 1024, "top_p": 0.8, "temperature": 0.8, "repetition_penalty": 1.1, "do_sample": True}
inp_text = tokenizer([text], return_tensors="pt").input_ids.cuda()

output = model.generate(inp_text, **gen_kwargs)
output_decoded = tokenizer.decode(output[0], skip_special_tokens=True)
output_decoded

## Statistic Data

In [ ]:
stats_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_all_3sigma_v9_from_v4_en_2023-11-18-15-00-17/data/en/datasets_en_stats.jsonl"
data_path = "/home/xiejunlin/workspace/Tianchi_FT-Data_Ranker/checkpoints/run/run_all_3sigma_v9_from_v4_en_2023-11-18-15-00-17/data/en/datasets_en.jsonl"
stat_reader = jsonlines.open(stats_path, 'r')
data_reader = jsonlines.open(data_path, 'r')

for stat, data in zip(stat_reader, data_reader):
    print(data)
    print(stat)
    break
    

In [17]:
data_path = "data/raw_data/raw_data_en.jsonl"
data_reader = jsonlines.open(data_path, 'r')
dataset_type = {}

for idx, line in enumerate(data_reader):
    ds_type = line['meta']['Dataset']
    tsk_type = line['meta']['Task']
    if dataset_type.get(ds_type, -1) == -1:
        dataset_type[ds_type] = 0
    dataset_type[ds_type] += 1


## Kmeans